In [1]:
# load packages
library(edgeR)
library(sva)
library(fgsea)
library(tidyr)
source('../ABCA7lof2//degs.r')
set.seed(5)

Loading required package: limma

Loading required package: mgcv

Loading required package: nlme

This is mgcv 1.8-40. For overview type 'help("mgcv-package")'.

Loading required package: genefilter

Loading required package: BiocParallel



In [106]:
# functions
compute_degs = function (counts.df, mod1, mod0,var, n.sv = NULL)
{
    dge <- DGEList(counts = counts.df)
    dge <- calcNormFactors(dge)

    v <- voom(dge, design = mod1)
    if (is.null(n.sv)) {
        n.sv <- num.sv(v$E, mod1, method = "be")
    }
    svobj <- sva(v$E, mod1, mod0, n.sv = n.sv)
    mod1 <- cbind(mod1, svobj$sv)
    v <- voom(dge, design = mod1)
    fit <- lmFit(v, design = mod1)
    fit <- eBayes(fit)
    #res1 <- topTable(fit, coef = "rs3752246", n = Inf, sort.by = "p")
    res1 <- topTable(fit, coef =var, n = Inf, sort.by = "p")
    return(list(res1 = res1, C = svobj$sv, mod=mod1))
}

In [75]:
# load Henne Holstege et al data
df = read.csv('../processed_data/holstege_et_al//41588_2022_1208_MOESM4_ESM.csv', skip = 2, header = TRUE)

# merge with ABCA7 variant info
variant_info = read.csv('../raw_data/ROSMAP_WGS/HIGHandMED_coding_annotations_syn11724057_subset.csv', check.names=FALSE)
variant_info$Variant = paste0(variant_info$CHROM,':',variant_info$POS,':',variant_info$REF_x, '>', variant_info$ALT_0)
df = df[,c('Variant', 'REVEL', 'LOF')]
var_info = merge(variant_info, df, by = 'Variant')
var_info = var_info[var_info$GENE=='ABCA7',]

genotypes = var_info[,unlist(lapply(names(var_info), function(x) sum(startsWith(x, c('ROS', 'MAP', 'SM')))))>0]
genotypes[genotypes=='0/0']=0
genotypes[genotypes=='0/1']=1
genotypes[genotypes=='1/0']=1
genotypes[genotypes=='1/1']=2
df = apply(genotypes,2, function(x){as.numeric(x)})

variant_categories = as.data.frame(colSums(df[var_info$LOF.y==1,]))

temp = as.numeric(unlist(genotypes[var_info$ID=='rs3752246',]))                                  
temp = -1*(temp-2)
variant_categories$rs3752246 = temp

Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”
Warning message in FUN(newX[, i], ...):
“NAs introduced by coercion”


In [76]:
# load our large snRNAseq dataset
data = readRDS('../processed_data/mathys_et_al/all_summed_cts.rds')

# add variant data to metadata
meta = data$meta
meta = cbind(meta, variant_categories[meta$WGSid,])

In [77]:
# remove individuals from our snRNAseq cohort
all_data = readRDS('../processed_data/single_cell//stats_input_data_0825.rds')

keep = !meta$projid%in%rownames(all_data$summary)
meta = meta[keep,]
ncells = data$ncells[keep]
counts = data$counts[,keep]

In [78]:
expressed = all_data$expressed10

In [79]:
# add batch info
batch_ids = read.csv('../../ADSubtypes/metadata/Fastq_paths_432_PFC_HM_updated_edited.csv')
batch_ids = batch_ids[!duplicated(batch_ids$projid),]
rownames(batch_ids) = batch_ids$projid
meta$seq_batch = batch_ids[rownames(meta),'Batch']

In [81]:
# filter out individuals with too few cells per celltype
keep = ncells>=10
counts_indexed = counts[,keep]
meta_keep = meta[keep,]
meta_keep$niareagansc = as.numeric(meta_keep$niareagansc)

# filter
keep = (rowSums(is.na(meta_keep))==0) & !(is.na(meta_keep[,'colSums(df[var_info$LOF.y == 1, ])']) | (meta_keep[,'colSums(df[var_info$LOF.y == 1, ])']>0)) #& meta_keep$niareagansc%in%(unlist(list('3','4')))
meta_keep2 = meta_keep[keep,]
meta_keep2$apoe = ifelse(meta_keep2$apoe_genotype%in%c(34,44,24), 1, 0)
counts_keep2 = counts_indexed[,keep]
counts_keep2 = counts_keep2[rownames(counts_keep2)%in%expressed$Ex & rowSums(counts_keep2>0)>15,]

In [82]:
meta_keep2$rs3752246_binary = ifelse(meta_keep2$rs3752246>0, 1, 0)

table(meta_keep2$rs3752246_binary)


  0   1 
227 133 

In [16]:
# compute DEGs rs3752246
# remove any LOF individuals

meta_keep2$rs3752246_binary = ifelse(meta_keep2$rs3752246>0, 1, 0)

mod1 <- model.matrix(~rs3752246_binary + amyloid + nft + msex + age_death + apoe + pmi + seq_batch + niareagansc, data = meta_keep2)
mod0 <- model.matrix(~ amyloid + nft + msex + age_death + apoe + pmi + seq_batch + niareagansc, data = meta_keep2)

out = compute_degs(counts_keep2, mod1, mod0, 'rs3752246_binary')
curr = out$res1
scores = sign(curr$logFC) * -log10(curr$P.Value)
names(scores) = rownames(curr)

Number of significant surrogate variables is:  21 
Iteration (out of 5 ):1  2  3  4  5  

In [623]:
# compute fgsea results rs3752246
paths = read.csv('../supplementary_tables/data_s8.csv')
paths = paths[paths$is_gene=='True',]
P = list()
for(i in unique(paths$cluster)){
    P[[as.character(i)]] = paths[paths$cluster==i,'description']
}      

o = fgsea(pathways = P, stats = scores, minSize = 5, maxSize = 1000, nproc=1, nPermSimple=10000)
o = o[order(o$pval,decreasing=T),]
o$score = sign(o$NES) * -log10(o$pval)
o$pathway = factor(o$pathway, levels = o$pathway)

In [634]:
write.csv(meta_keep2[, c('projid', 'WGSid', 'rs3752246_binary', 'amyloid', 'nft', 'msex', 'age_death', 'apoe', 'pmi', 'seq_batch', 'niareagansc')], '../processed_data/mathys_et_al/data_for_carles.csv', row.names=FALSE)

In [636]:
x = o[order(o$pval,decreasing=FALSE),]
x$leadingEdge = NULL

write.csv(x, '../processed_data/common_var/rs3755246_cluster_effects.csv')
write.csv(as.data.frame(scores), '../processed_data/common_var/rs3752246_binary_scores.csv')